In [ ]:
import os

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from pytorch_lightning.callbacks import EarlyStopping

from pina import Condition, Span, LabelTensor, Trainer
from pina.problem import TimeDependentProblem, SpatialProblem, ParametricProblem
from pina.operators import grad
from pina.model import FeedForward
from pina.solver import PINN

import optuna

In [ ]:
input_filepath = "/kaggle/input/cleaned-oxi3"
file_paths = [os.path.join(input_filepath, fname) for fname in os.listdir(input_filepath)]

In [ ]:
class OxidationDataLoader(Dataset):
    def __init__(self, file_paths):
        # I am creating a list of all dfs
        dfs = []
        for f in file_paths:
            df = pd.read_csv(f)
            dfs.append(df)

        self.df = pd.concat(dfs, ignore_index = True)
        # I am converting everything to numbers from strings like 1e15 etc
        self.df['X'] = pd.to_numeric(self.df['X'], errors = 'coerce')
        self.df['Y'] = pd.to_numeric(self.df['Y'], errors = 'coerce')
        self.df['Time'] = pd.to_numeric(self.df['Time'], errors='coerce')
        self.df['O2 Flow'] = pd.to_numeric(self.df['O2 Flow'], errors='coerce')
        self.df['N2 Flow'] = pd.to_numeric(self.df['N2 Flow'], errors='coerce')
        self.df['Temperature'] = pd.to_numeric(self.df['Temperature'], errors='coerce')

        self.df.dropna(subset = ['X', 'Y', 'Time', 'O2 Flow', 'N2 Flow', 'Temperature'], inplace = True)

        epsilon = 1e-9 # to avoid -infi
        self.df.loc[self.df['Y'] <= 0, 'Y'] = epsilon

        # I am converting Y outputs to log Y because only 4% of total dataset
        # has oxidation conc values around 96% has only bulk pure Si
        # to in order to not let the model simply memorise bulk we do
        self.df['logY'] = np.log10(self.df['Y'])

        # logY > 2 --> Y > 100
        mask_reactive = self.df['logY'] > 2.0
        # two different dfs for reacted part and bulk part
        df_reactive = self.df[mask_reactive]
        df_bulk = self.df[~mask_reactive]

        df_bulk = df_bulk.iloc[:10]        # only 10 data points after si sio2 interface
        # n_reactive = len(df_reactive)
        # if len(df_bulk) > n_reactive:
        #     df_bulk = df_bulk.sample(n = n_reactive, random_state = 0)

        self.df_balanced = pd.concat([df_reactive, df_bulk]).reset_index(drop = True)
        self.df_balanced = self.df_balanced.sample(frac = 1, random_state = 0).reset_index(drop = True)
        # we are dropping more parts of bulk so that now
        # the oxidation concentrated part is not minority and mmodel doesnt overfit

        features = self.df_balanced[['X', 'Time', 'O2 Flow', 'N2 Flow', 'Temperature']].values
        targets = self.df_balanced['logY'].values

        self.inputs = torch.tensor(features, dtype = torch.float32)
        self.targets = torch.tensor(targets, dtype = torch.float32).unsqueeze(-1)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [ ]:
class OxidationPhysics(TimeDependentProblem, SpatialProblem, ParametricProblem):
    def __init__(self, data_loader):
        super().__init__()

        df = data_loader.df_balanced

        x_min, x_max = df['X'].min(), df['X'].max()
        self.spatial_domain = Span({'x': [x_min, x_max]})

        t_min, t_max = df['Time'].min(), df['Time'].max()
        self.temporal_domain = Span({'t': [t_min, t_max]})

        self.params_domain = Span({
            'o2': [df['O2 Flow'].min(), df['O2 Flow'].max()],
            'n2': [df['N2 Flow'].min(), df['N2 Flow'].max()],
            'temp': [df['Temperature'].min(), df['Temperature'].max()]
        })

        labeled_inputs = LabelTensor(
            data_loader.inputs,
            ['x', 't', 'o2', 'n2', 'temp']
        )

        labeled_targets = LabelTensor(
            data_loader.targets,
            ['u']
        )

        self.conditions = {
            'data': Condition(
                input_points = labeled_inputs,
                output_points = labeled_targets
            ),

            'physics': Condition(
                equation = self.diffusion_equation,

                location = Span({
                    'x': [x_min, x_max], 
                    't': [t_min, t_max], 
                    'o2': [df['O2 Flow'].min(), df['O2 Flow'].max()], 
                    'n2': [df['N2 Flow'].min(), df['N2 Flow'].max()], 
                    'temp': [df['Temperature'].min(), df['Temperature'].max()]
                })
            ),

            'surface': Condition(
                equation = self.surface_bc,

                location = Span({
                    'x': [0, 1e-6], # Very close to surface
                    't': [t_min, t_max],
                    'o2': [df['O2 Flow'].min(), df['O2 Flow'].max()],
                    'n2': [df['N2 Flow'].min(), df['N2 Flow'].max()],
                    'temp': [df['Temperature'].min(), df['Temperature'].max()]
                })
            ),
        }

    def diffusion_equation(self, input_, output_):
        """
        The Physics: u_t - D * u_xx = 0
        """

        u_t = grad(output_, input_, d = 't')
        u_xx = grad(output_, input_, d = 'x', dd = 'x')

        # Physics Logic: Diffusivity D depends on Temperature
        # D ~ Temperature / 1000 (Scaled approximate)
        
        temp = input_.extract(['temp'])
        D = 1e-4 * (temp/1000.0)

        return u_t - (D * u_xx)

    def surface_bc(self, input_, output_):
        """
        The Boundary: u(surface) approx 14 * O2_Flow
        """

        u = output_.extract(['u'])
        o2 = input_.extract(['o2'])

        return u - (14.0 * o2) # 14.0 must actually be a trainable param
# this is approximation for partial pressure we will include henry's law here

In [ ]:
def create_model(input_dim: int, output_dim: int,
                 n_layers: int, n_neurons: int, activation: str = 'tanh') -> torch.nn.Module:

    layers = []

    layers.append({
        'input_dim': input_dim,
        'output_dim': n_neurons,
        'activation': activation
    })

    for _ in range(n_layers - 1):
        layers.append({
            'input_dim': n_neurons,
            'output_dim': n_neurons,
            'activation': activation
        })

    layers.append({
        'input_dim': n_neurons,
        'output_dim': output_dim,
        'activation': None
    })

    model = FeedForward(
        layers = layers,
        input_variables = ['x', 't', 'o2', 'n2', 'temp'],
        output_variables = ['u']
    )

    return model

In [ ]:
def objective(trial):
    n_layers = trial.suggest_int('n_layers', 3, 6)
    n_neurons = trial.suggest_int('n_neurons', 32, 128)
    activation = trial.suggest_categorical('activation', ['tanh', 'sigmoid'])

    lr = trial.suggest_float('learning_rate', 1e-4, 1e-2, log = True)

    w_data = trial.suggest_float('weight_data', 1.0, 20.0)
    w_phys = trial.suggest_float('weight_phys', 0.1, 5.0)
    w_surf = trial.suggest_float('weight_surf', 1.0, 10.0)

    problem = OxidationPhysics(data_loader = loader)

    problem.conditions['data'].weight = w_data
    problem.conditions['physics'].weight = w_phys
    problem.conditions['surface'].weight = w_surf

    problem.discretise_domain(
        n = 2000,
        mode = 'random',
        locations = ['physics', 'surface']
    )

    model = create_model(
        input_dim=5, 
        output_dim=1, 
        n_layers=n_layers, 
        n_neurons=n_neurons, 
        activation=activation
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    solver = PINN(
        problem=problem,
        model=model,
        optimizer=optimizer
    )

    trainer = Trainer(
        solver=solver,
        max_epochs=40,
        accelerator='auto',
        logger=False,
        enable_progress_bar=False,
        enable_model_summary=False
    )

    trainer.train()

    final_loss = trainer.logged_metrics.get('mean_loss', None)

    if final_loss is None:
        final_loss = trainer.logged_metrics.get('loss', 1e9)
        
    return final_loss.item()

In [ ]:
def train_best_model(best_params, data_loader, epochs = 500, output_dir = 'results'):
    print(f"Starting final training for {epochs} epochs")

    problem = OxidationPhysics(data_loader)

    problem.conditions['data'].weight = best_params.get('weight_data', 10.0)
    problem.conditions['physics'].weight = best_params.get('weight_phys', 1.0)
    problem.conditions['surface'].weight = best_params.get('weight_surf', 5.0)

    problem.discretise_domain(
        n=4000, 
        mode='random', 
        locations=['physics', 'surface']
    )

    model = create_model(
        input_dim=5, 
        output_dim=1, 
        n_layers=best_params['n_layers'], 
        n_neurons=best_params['n_neurons'], 
        activation=best_params.get('activation', 'tanh')
    )

    optimizer = torch.optim.Adam(
        model.parameters(), 
        lr=best_params['learning_rate']
    )

    solver = PINN(
        problem=problem,
        model=model,
        optimizer=optimizer
    )

    early_stop_callback = EarlyStopping(
        monitor = "mean_loss",
        min_delta = 1e-4,
        patience = 50,
        verbose = True,
        mode = "min"
    )

    trainer = Trainer(
        solver=solver,
        max_epochs=epochs,
        accelerator='auto',
        default_root_dir=output_dir,
        enable_progress_bar=True,
        enable_model_summary=True,
        callbacks = [early_stop_callback]
    )

    trainer.train()
    print("Training Complete")

    final_train_loss = trainer.logged_metrics.get('mean_loss', torch.tensor(-1.0)).item()
    print(f"Final Training Loss: {final_train_loss:.6f}")

    os.makedirs(output_dir, exist_ok = True)
    save_path = os.path.join(output_dir, "best_model.pth")
    torch.save(solver.model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

    return solver, trainer, final_train_loss

In [ ]:
def evaluate_performance(solver, data_loader, train_loss = None):
    print("Evaluating model performance")

    device = solver.device if hasattr(solver, 'device') else 'cpu'

    inputs = data_loader.inputs.to(device)
    true_log_u = data_loader.targets.cpu().numpy()
    
    solver.model.eval()
    with torch.no_grad():
        preds_log_u = solver.model(inputs).cpu().numpy()

    mask_react = true_log_u > 2.0
    mse_log_react = mean_squared_error(true_log_u[mask_react], preds_log_u[mask_react])
    rmse_log_react = np.sqrt(mse_log_react)

    mse_log_bulk = mean_squared_error(true_log_u[~mask_react], preds_log_u[~mask_react])
    rmse_log_bulk = np.sqrt(mse_log_bulk)
    
    true_Y = np.power(10, true_log_u)
    pred_Y = np.power(10, preds_log_u)

    relative_error = np.abs((true_Y - pred_Y) / (true_Y + 1e-9))
    median_rel_error = np.median(relative_error) * 100
    mean_rel_error = np.mean(relative_error) * 100

    print(f"   Reaction Zone RMSE (Log): {rmse_log_react:.4f}")
    print(f"   Bulk Zone RMSE (Log):     {rmse_log_bulk:.4f}")
    print(f"   Median Rel Error (Y):     {median_rel_error:.2f}%")

    if train_loss is not None:
        total_rmse = np.sqrt(mean_squared_error(true_log_u, preds_log_u))
        print(f"   Gap (Eval RMSE - Train Loss): {total_rmse - train_loss:.4f}")
    
    return rmse_log_react, rmse_log_bulk, median_rel_error